_Anmerkung: Ggf. müssen sie nach dem Öffnen des Notebooks noch einmal auf 'Kernel > Restart & Run All' clicken_

<h1>Abfragen mit SPARQL</h1>

In diesem Demonstrator werden wir zeigen, wie SPARQL Abfragen genutzt werden können, 
um aus Open Data Beständen weiteres Wissen zu ziehen.

![SPARQL Logo](./media/SPARQL.png)

Zuerst: Was ist SPARQL?
Bei SPARQL handelt es sich um die **S**PARQL **P**rotocol **A**nd **R**DF **Q**uery **L**anguage.
Es ist also sowohl ein Protokoll als auch eine Abfragesprache für das Resource Description Framework. 
Während SQL die Abfragesprache für relationale Datenbanken ist, ist SPARQL die geläufige Abfragesprache für RDF-Daten, welche nicht als Tabellen sondern als Graph mit gelabelten Kanten zu verstehen sind.

Eine SPARQL Abfrage ähnelt in ihren Augen möglicherweise einer SQL-Abfrage, jedoch sind die Gemeinsamkeiten nur oberflächlich.
Abfragen besitzen einen SELECT-Part, der die resultierenden Variablen (von der Struktur den Spalten in SQL entsprechend) wiedergibt und einem WHERE-Part, der definiert, welche Eigenschaften die resultierenden Filme alle gemeinsam besitzen.

In [1]:
from data.sparql import query, table

table(query("""
SELECT ?movie
WHERE
{
 ?movie wdt:P31 wd:Q11424
}
LIMIT 10
"""))

movie
http://www.wikidata.org/entity/Q372
http://www.wikidata.org/entity/Q593
http://www.wikidata.org/entity/Q595
http://www.wikidata.org/entity/Q1365
http://www.wikidata.org/entity/Q2201
http://www.wikidata.org/entity/Q2345
http://www.wikidata.org/entity/Q2875
http://www.wikidata.org/entity/Q3092
http://www.wikidata.org/entity/Q3187
http://www.wikidata.org/entity/Q3208


Diese Query liefert die ersten 10 Filme, welche in Wikidata gelistet sind.
Zwar können sie auf die Einträge klicken, um zu sehen, um was es sich handelt, jedoch ist dieses Format noch nicht sonderlich leserlich.

Um statt der Objekte selbst ihre Label zu erhalten, erweitern wir die vorherige Query. Diesmal soll nicht der Film selbst, sondern sein Titel angezeigt werden.

In [2]:
from data.sparql import query, table

table(query(
"""
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424 .
 ?movie wdt:P1448 ?title       
} 
LIMIT 10
"""
))    

title
...men Olsenbanden var ikke død
Le guerriere dal seno nudo
La vendetta dei gladiatori
Zinebi Festival Internacional de Cine Documental y Cortometraje de Bilbao
Relatives
Börn
Syurpriz
Petualang Tak Kenal Menyerah
طلق صناعي
Brexit (2019)


Wie sie sehen, muss nicht jede Variable auch ausgegeben werden.
Hier sieht man nur den Titel, aber nicht das Film-Objekt.

Zudem ist dies die erste Query, bei der mehrere Kriterien (nämlich Objekt ?movie ist vom Typ Film und Object ?movie hat Titel ?title.
Jedes dieser logischen Tripletts wird durch einen Punkt vom nächsten getrennt.
Um etwas Schreibarbeit zu sparen, kann auch 

```
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424;
        wdt:P1448 ?title       
} 
LIMIT 10
```

schreiben. Hier wird durch das Semikolon ";" das Subjekt aus dem letzten Triplet für das Nächste wiederverwendet.

Schreiben sie ebenfalls eine Abfrage, welche die IMDB ID dreier Filme ausgibt.
Das Prädikat für 'hat IMDB ID' lautet 'wdt:P345', bei der IMDB ID handelt es sich um eine ID der 'Internet Movie Database'.

In [3]:
# Tragen sie hier ihren SPARQL-Code ein und klicken sie auf RUN, um ihn auszutesten.
# Denken sie daran, ggf. über 'from data.sparql import query, table' die notwendigen Funktionen zu importieren,
# sollte es sie hier zu einem Fehler kommen.








<h2> Anreicherung lokaler Daten mit Daten aus dem Netz </h2>

Die durch SPARQL verfügbar gemachten Daten können nun verwendet werden, 
um beispielsweise zusätzliche Informationen zu denen einer Datenbank hinzuzufügen.
In diesem Notebook handelt es sich hierbei um eine MongoDB, welche eine vielzahl an Filmen enthält.

In [4]:
import data.movies as movies

iron_man_movies = movies.get_by_title('Iron Man')

table(iron_man_movies)

Key,Value
index,12588
adult,False
belongs_to_collection,Iron Man Collection
budget,140000000
genres,"Action,Science Fiction,Adventure"
homepage,http://www.ironmanmovie.com/
id,1726
imdb_id,tt0371746
original_language,en
original_title,Iron Man


Um die in der Datenbank befindlichen Einträge zu Filmen um zusätzliche Informationen zu erweitern, muss zunächst ein geeigneter eindeutiger Identifier für einen Film gefunden werden. Dazu eignen sich einige Einträge eines Films besser als andere.

Frage: Welche Eigenschaft eines Filmes haltet ihr für sinvoll um online nach Infos zu suchen und warum?





<h3> Suche nach Namen: </h3>

In [5]:
from data.sparql import query, data, table, entity_to_json

#movie is instance (P31) of film (Q11424)
#movie has label ?titel

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
"""
))


movie,title
http://www.wikidata.org/entity/Q185884,Tetsuo: The Iron Man
http://www.wikidata.org/entity/Q192724,Iron Man
http://www.wikidata.org/entity/Q192724,Iron Man
http://www.wikidata.org/entity/Q192724,Iron Man
http://www.wikidata.org/entity/Q205028,Iron Man 2
http://www.wikidata.org/entity/Q205028,Iron Man 2
http://www.wikidata.org/entity/Q205028,Iron Man 2
http://www.wikidata.org/entity/Q209538,Iron Man 3
http://www.wikidata.org/entity/Q209538,Iron Man 3
http://www.wikidata.org/entity/Q209538,Iron Man 3


<h3> Suche nach imdb_id: </h3>

In [6]:
from data.sparql import query, table

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        wdt:P345 ?imdb_id .
 FILTER CONTAINS (?imdb_id, "tt0371746")
} 
LIMIT 1
"""
))



movie,imdb_id
http://www.wikidata.org/entity/Q192724,tt0371746


Über den Namen des Films oder andere Attribute wie die IMDB-ID können weitere Informationen gesucht werden:

In [3]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

# a bit hacky, but it retrieves the id from the wikidata URI, which has the form https://www.wikidata.org/wiki/{id}
entity_id = data(result)[0]['movie'].split("/").pop()

entity_to_json(entity_id)

{'entities': {'Q185884': {'pageid': 184692,
   'ns': 0,
   'title': 'Q185884',
   'lastrevid': 1432440153,
   'modified': '2021-05-31T15:58:32Z',
   'type': 'item',
   'id': 'Q185884',
   'labels': {'de': {'language': 'de', 'value': 'Tetsuo: The Iron Man'},
    'en': {'language': 'en', 'value': 'Tetsuo: The Iron Man'},
    'es': {'language': 'es', 'value': 'Tetsuo: The Iron Man'},
    'fr': {'language': 'fr', 'value': 'Tetsuo'},
    'it': {'language': 'it', 'value': 'Tetsuo'},
    'ja': {'language': 'ja', 'value': '鉄男'},
    'lv': {'language': 'lv', 'value': 'Tecuo: Dzelzs cilvēks'},
    'pl': {'language': 'pl', 'value': 'Tetsuo: The Iron Man'},
    'ru': {'language': 'ru', 'value': 'Тэцуо — железный человек'},
    'nl': {'language': 'nl', 'value': 'Tetsuo: The Iron Man'},
    'eml': {'language': 'eml', 'value': 'Tetsuo'},
    'fa': {'language': 'fa', 'value': 'تتسوئو: مرد آهنین'},
    'pt': {'language': 'pt', 'value': 'Tetsuo'},
    'zh': {'language': 'zh', 'value': '鐵男'},
    'uk': {

Hier erkennt man, dass wir eine große Menge an Daten zum Film erhalten. 
Wir können hier den Film-Titel in vielen Sprachen angeben, herausfinden, welcher Direktor für den Schnitt des Filmes verantwortlich ist, welche unterschiedlichen Jugendschutz-Label ein Film hat, etc.

<h3> Einfügen aller Schauspieler eines Films </h3>

In [8]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

Einige der Eigenschaften sind jedoch über ihre Wikidata-ID aufgelistet. Diese muss aufgelöst werden, um menschenverständliche Daten zu erhalten, also z.B. das es sich bei 'P345' um eine 'IMDB ID' handelt. 

In [9]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746".
    # take all claims on this movie
    ?title ?property ?cast .
    # that involve a human
    ?cast wdt:P31 wd:Q5 .

    # get the property label
    # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
    hint:Query hint:optimizer "None" .
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)

table (result)

titleLabel,castLabel,property,propLabel
Iron Man,Jon Favreau,http://www.wikidata.org/prop/direct/P57,director
Iron Man,Mark Fergus,http://www.wikidata.org/prop/direct/P58,screenwriter
Iron Man,Hawk Ostby,http://www.wikidata.org/prop/direct/P58,screenwriter
Iron Man,Art Marcum,http://www.wikidata.org/prop/direct/P58,screenwriter
Iron Man,Matt Holloway,http://www.wikidata.org/prop/direct/P58,screenwriter
Iron Man,Ramin Djawadi,http://www.wikidata.org/prop/direct/P86,composer
Iron Man,Gwyneth Paltrow,http://www.wikidata.org/prop/direct/P161,cast member
Iron Man,Clark Gregg,http://www.wikidata.org/prop/direct/P161,cast member
Iron Man,Shaun Toub,http://www.wikidata.org/prop/direct/P161,cast member
Iron Man,Robert Downey Jr.,http://www.wikidata.org/prop/direct/P161,cast member


Die so abgerufene Liste an Schauspielern bzw. Mitwirkenden dieses Films wird nun dem Datensatz der lokalen Datenbank hinzugefügt. Dazu werden die Daten zunächst aufbereitet und dann in dem Datensatz gespeichert.

In [ ]:
from data.sparql import query, data, table, entity_to_json
import data.movies as movies


result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746".
    # take all claims on this movie
    ?title ?property ?cast .
    # that involve a human
    ?cast wdt:P31 wd:Q5 .

    # get the property label
    # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
    hint:Query hint:optimizer "None" .
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)
table(result)
dict = {"cast_members": [{"name": cast["castLabel"]["value"], "role": cast["propLabel"]["value"]} for cast in result["results"]["bindings"]]}
#print (dict)



movies.add_data_by_title("Iron Man", dict)
iron_man = movies.get_by_title('Iron Man')
table(iron_man)

Dieser Vorgang kann nun automatisiert für alle Einträge der lokalen Datenbank durchlaufen werden. 

In [ ]:
from data.sparql import query, data, table, entity_to_json
import json
import data.movies as movies

def wikidata_query (imdb_id):
    return query(
    f"""
    SELECT ?titleLabel ?castLabel ?property ?propLabel
    WHERE {{
        ?title wdt:P345 "{imdb_id}".
        # take all claims on this movie
        ?title ?property ?cast .
        # that involve a human
        ?cast wdt:P31 wd:Q5 .

        # get the property label
        # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
        hint:Query hint:optimizer "None" .
        ?prop wikibase:directClaim ?property .

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    )

def extract_cast (wikidata_result):
    dict = {"cast_members": []}
    for cast_member in result["results"]["bindings"]:
        actor = {
            "Name": cast_member["castLabel"]["value"],
            "Role": cast_member["propLabel"]["value"]
        }
        dict["cast_members"].append(actor)
    return dict


#load all movies from local db (limited to 5):
local_movies = movies.get_all(10000)
for movie in local_movies:
    imdb_id = movie["imdb_id"]
    title = movie["original_title"]
    movie_id = movie["id"]
 
    #Query an 
    result = wikidata_query(imdb_id)
    
    cast_dict = extract_cast(result)
    
    movies.add_data_by_title(title, cast_dict)
    #print(json.dumps(movies.get_by_title(title), indent=1))
    
    print (f"handled movie: '{title}' with imdb {imdb_id}")



handled movie: 'Toy Story' with imdb tt0114709
handled movie: 'Jumanji' with imdb tt0113497
handled movie: 'Grumpier Old Men' with imdb tt0113228
handled movie: 'Waiting to Exhale' with imdb tt0114885
handled movie: 'Father of the Bride Part II' with imdb tt0113041
handled movie: 'Heat' with imdb tt0113277
handled movie: 'Sabrina' with imdb tt0114319
handled movie: 'Tom and Huck' with imdb tt0112302
handled movie: 'Sudden Death' with imdb tt0114576
handled movie: 'GoldenEye' with imdb tt0113189
handled movie: 'The American President' with imdb tt0112346
handled movie: 'Dracula: Dead and Loving It' with imdb tt0112896
handled movie: 'Balto' with imdb tt0112453
handled movie: 'Nixon' with imdb tt0113987
handled movie: 'Cutthroat Island' with imdb tt0112760
handled movie: 'Casino' with imdb tt0112641
handled movie: 'Sense and Sensibility' with imdb tt0114388
handled movie: 'Four Rooms' with imdb tt0113101
handled movie: 'Ace Ventura: When Nature Calls' with imdb tt0112281
handled movie: '

handled movie: 'Casper' with imdb tt0112642
handled movie: 'Clockers' with imdb tt0112688
handled movie: 'Congo' with imdb tt0112715
handled movie: 'Crimson Tide' with imdb tt0112740
handled movie: 'Crumb' with imdb tt0109508
handled movie: 'Desperado' with imdb tt0112851
handled movie: 'Devil in a Blue Dress' with imdb tt0112857
handled movie: 'Die Hard: With a Vengeance' with imdb tt0112864
handled movie: 'The Doom Generation' with imdb tt0112887
handled movie: 'Feast of July' with imdb tt0113044
handled movie: 'First Knight' with imdb tt0113071
handled movie: 'Free Willy 2 - The Adventure Home' with imdb tt0113114
handled movie: 'Hackers' with imdb tt0113243
handled movie: 'Jeffrey' with imdb tt0113464
handled movie: 'Johnny Mnemonic' with imdb tt0113481
handled movie: 'Judge Dredd' with imdb tt0113492
handled movie: 'Jury Duty' with imdb tt0113500
handled movie: 'Kids' with imdb tt0113540
handled movie: 'Living in Oblivion' with imdb tt0113677
handled movie: 'Lord of Illusions' wit

handled movie: 'Stargate' with imdb tt0111282
handled movie: 'The Santa Clause' with imdb tt0111070
handled movie: 'The Shawshank Redemption' with imdb tt0111161
handled movie: 'Shallow Grave' with imdb tt0111149
handled movie: 'Suture' with imdb tt0108260
handled movie: 'Fresa y chocolate' with imdb tt0106966
handled movie: 'Swimming with Sharks' with imdb tt0114594
handled movie: 'The Sum of Us' with imdb tt0111309
handled movie: 'Senior Trip' with imdb tt0113936
handled movie: '活着' with imdb tt0110081
handled movie: 'Tank Girl' with imdb tt0114614
handled movie: 'Tales from the Crypt: Demon Knight' with imdb tt0114608
handled movie: 'Star Trek: Generations' with imdb tt0111280
handled movie: 'Tales from the Hood' with imdb tt0114609
handled movie: 'Tom & Viv' with imdb tt0111454
handled movie: 'Village of the Damned' with imdb tt0114852
handled movie: 'Tommy Boy' with imdb tt0114694
handled movie: 'Vanya on 42nd Street' with imdb tt0111590
handled movie: 'The Underneath' with imdb t

handled movie: 'House Party 3' with imdb tt0110064
handled movie: 'The Hudsucker Proxy' with imdb tt0110074
handled movie: 'I'll Do Anything' with imdb tt0110097
handled movie: 'In the Army Now' with imdb tt0110123
handled movie: 'In the Line of Fire' with imdb tt0107206
handled movie: 'In the Name of the Father' with imdb tt0107207
handled movie: 'The Inkwell' with imdb tt0110137
handled movie: 'What's Love Got to Do with It' with imdb tt0108551
handled movie: 'Jimmy Hollywood' with imdb tt0110197
handled movie: 'Judgment Night' with imdb tt0107286
handled movie: 'Jurassic Park' with imdb tt0107290
handled movie: 'Kalifornia' with imdb tt0107302
handled movie: 'Killing Zoe' with imdb tt0110265
handled movie: 'King of the Hill' with imdb tt0107322
handled movie: 'Lassie' with imdb tt0110305
handled movie: 'Last Action Hero' with imdb tt0107362
handled movie: 'Life With Mikey' with imdb tt0107413
handled movie: 'Lightning Jack' with imdb tt0110353
handled movie: 'M. Butterfly' with imdb

handled movie: 'Rude' with imdb tt0114305
handled movie: 'Carried Away' with imdb tt0115837
handled movie: 'All Dogs Go to Heaven 2' with imdb tt0115509
handled movie: 'Land and Freedom' with imdb tt0114671
handled movie: 'Denise Calls Up' with imdb tt0112844
handled movie: 'Theodore Rex' with imdb tt0114658
handled movie: 'A Family Thing' with imdb tt0116275
handled movie: 'Frisk' with imdb tt0113122
handled movie: 'Sgt. Bilko' with imdb tt0117608
handled movie: 'Jack & Sarah' with imdb tt0113448
handled movie: 'Girl 6' with imdb tt0116414
handled movie: 'Diabolique' with imdb tt0116095
handled movie: 'Un indien dans la ville' with imdb tt0111543
handled movie: 'Roula - Dunkle Geheimnisse' with imdb tt0117517
handled movie: 'Peanuts – Die Bank zahlt alles' with imdb tt0117312
handled movie: 'Happy Weekend' with imdb tt0116485
handled movie: 'Nelly & Monsieur Arnaud' with imdb tt0113947
handled movie: 'Courage Under Fire' with imdb tt0115956
handled movie: 'Mission: Impossible' with im

...

handled movie: 'Viskningar och rop' with imdb tt0069467
handled movie: 'The Entertainer' with imdb tt0053796
handled movie: 'Il giardino dei Finzi Contini' with imdb tt0065777
handled movie: 'Howling III: The Marsupials' with imdb tt0093227
handled movie: 'Kiss Me Deadly' with imdb tt0048261
handled movie: 'The Lion in Winter' with imdb tt0063227
handled movie: 'The Misfits' with imdb tt0055184
handled movie: 'Moby Dick' with imdb tt0049513
handled movie: 'Popcorn' with imdb tt0102690
handled movie: 'Rembrandt' with imdb tt0028167
handled movie: 'Sweet Smell of Success' with imdb tt0051036
handled movie: 'Written on the Wind' with imdb tt0049966
handled movie: 'La decima vittima' with imdb tt0059095
handled movie: 'Oberst Redl' with imdb tt0089716
handled movie: 'Obsession' with imdb tt0074991
handled movie: 'Suspiria' with imdb tt0076786
handled movie: '精武門' with imdb tt0068767
handled movie: 'Christiane F. - Wir Kinder vom Bahnhof Zoo' with imdb tt0082176
handled movie: '唐山大兄' with i

handled movie: 'Casualties of War' with imdb tt0097027
handled movie: 'Chances Are' with imdb tt0097044
handled movie: 'Cookie' with imdb tt0097109
handled movie: 'Cyborg' with imdb tt0097138
handled movie: 'Dad' with imdb tt0097142
handled movie: 'Dead Bang' with imdb tt0097166
handled movie: 'Disorganized Crime' with imdb tt0097211
handled movie: 'Dream a Little Dream' with imdb tt0097236
handled movie: 'The Dream Team' with imdb tt0097235
handled movie: 'A Dry White Season' with imdb tt0097243
handled movie: 'Earth Girls Are Easy' with imdb tt0097257
handled movie: 'Eddie and the Cruisers II: Eddie Lives!' with imdb tt0097262
handled movie: 'Eddie and the Cruisers' with imdb tt0085475
handled movie: 'Enemies: A Love Story' with imdb tt0097276
handled movie: 'Erik the Viking' with imdb tt0097289
handled movie: 'The Experts' with imdb tt0097314
handled movie: 'Family Business' with imdb tt0097328
handled movie: 'Farewell to the King' with imdb tt0097334
handled movie: 'Fat Man and Lit

handled movie: 'Jay and Silent Bob Strike Back' with imdb tt0261392
handled movie: 'Ghosts of Mars' with imdb tt0228333
handled movie: 'Summer Catch' with imdb tt0234829
handled movie: 'An American Rhapsody' with imdb tt0221799
handled movie: 'Happy Accidents' with imdb tt0208196
handled movie: 'Lisa Picard Is Famous' with imdb tt0233699
handled movie: 'Maybe Baby' with imdb tt0206926
handled movie: 'Tillsammans' with imdb tt0203166
handled movie: 'Punks' with imdb tt0160710
handled movie: 'Tortilla Soup' with imdb tt0255653
handled movie: 'Jeepers Creepers' with imdb tt0263488
handled movie: 'O' with imdb tt0184791
handled movie: 'Waking Up in Reno' with imdb tt0219400
handled movie: 'Speedway Junky' with imdb tt0155197
handled movie: '3 Ninjas' with imdb tt0103596
handled movie: '3 Ninjas Kick Back' with imdb tt0109015
handled movie: '3 Ninjas Knuckle Up' with imdb tt0112255
handled movie: 'The Hunter' with imdb tt0080907
handled movie: 'Maniac' with imdb tt0081114
handled movie: 'Va

handled movie: 'Tape' with imdb tt0275719
handled movie: 'Heist' with imdb tt0252503
handled movie: 'Shallow Hal' with imdb tt0256380
handled movie: 'King of the Jungle' with imdb tt0177888
handled movie: 'Maze' with imdb tt0246072
handled movie: 'When a Stranger Calls' with imdb tt0080130
handled movie: 'Timerider: The Adventure of Lyle Swann' with imdb tt0086443
handled movie: 'The Wash' with imdb tt0290332
handled movie: 'Harry Potter and the Philosopher's Stone' with imdb tt0241527
handled movie: 'The Fluffer' with imdb tt0245115
handled movie: 'Novocaine' with imdb tt0234354
handled movie: 'Black Knight' with imdb tt0265087
handled movie: 'Out Cold' with imdb tt0253798
handled movie: 'Spy Game' with imdb tt0266987
handled movie: 'El espinazo del diablo' with imdb tt0256009
handled movie: 'In the Bedroom' with imdb tt0247425
handled movie: 'Così ridevano' with imdb tt0139951
handled movie: 'Hell Up In Harlem' with imdb tt0070169
handled movie: 'Bucktown' with imdb tt0072747
handled

handled movie: 'Fire and Ice' with imdb tt0085542
handled movie: 'Forbidden Zone' with imdb tt0080752
handled movie: 'The Formula' with imdb tt0080754
handled movie: 'Foxes' with imdb tt0080756
handled movie: 'Galaxina' with imdb tt0080771
handled movie: 'Impostor' with imdb tt0160399
handled movie: 'Kung Pow: Enter the Fist' with imdb tt0240468
handled movie: 'Snow Dogs' with imdb tt0281373
handled movie: '48 Hrs.' with imdb tt0083511
handled movie: 'Abschied - Brechts letzter Sommer' with imdb tt0206608
handled movie: 'Italiensk for begyndere' with imdb tt0243862
handled movie: 'Ha-Hesder' with imdb tt0268696
handled movie: 'Blankman' with imdb tt0109288
handled movie: 'Brainstorm' with imdb tt0085271
handled movie: 'Dragon: The Bruce Lee Story' with imdb tt0106770
handled movie: 'Jeder für sich und Gott gegen alle' with imdb tt0071691
handled movie: '4 for Texas' with imdb tt0057071
handled movie: 'Herz aus Glas' with imdb tt0074626
handled movie: 'Little Dieter Needs to Fly' with i

In [ ]:
import json
import data.movies as movies

print(json.dumps(movies.get_by_title("Dingo"), indent=1))




In [ ]:
Frage: Wie kann man hier die Performance erhöhen

Zu welchen problemen kann es kommen und wie kann auf diese reagiert werden?

Frage:
Welche Daten kann man nicht einfach über Wikidata beziehen?
Bsp. 
- Wie lange liefen Filme in einem Kino?
- In Welchen Kinos lief ein Film?